# Tuning PassiveAggressiveRegressor

In [1]:
import numpy
import csv
import time
import pandas
import signal
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

In [3]:
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import MultiOutputClassifier

from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.ensemble.weight_boosting import AdaBoostRegressor
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.bagging import BaggingRegressor
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.ensemble.forest import ExtraTreesRegressor
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.ensemble.forest import RandomForestRegressor

from sklearn.linear_model.bayes import ARDRegression
from sklearn.linear_model.bayes import BayesianRidge
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_decomposition.cca_ import CCA
from sklearn.tree.tree import DecisionTreeClassifier
from sklearn.tree.tree import DecisionTreeRegressor
from sklearn.linear_model.coordinate_descent import ElasticNet
from sklearn.tree.tree import ExtraTreeClassifier
from sklearn.tree.tree import ExtraTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.gaussian_process.gpr import GaussianProcessRegressor
from sklearn.linear_model.huber import HuberRegressor
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.neighbors.regression import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.semi_supervised.label_propagation import LabelPropagation
from sklearn.semi_supervised.label_propagation import LabelSpreading
from sklearn.linear_model.least_angle import Lars
from sklearn.linear_model.coordinate_descent import Lasso
from sklearn.linear_model.least_angle import LassoLars
from sklearn.linear_model.least_angle import LassoLarsIC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model.base import LinearRegression
from sklearn.svm.classes import LinearSVC
from sklearn.svm.classes import LinearSVR
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.neural_network.multilayer_perceptron import MLPRegressor
from sklearn.linear_model.coordinate_descent import MultiTaskElasticNet
from sklearn.linear_model.coordinate_descent import MultiTaskLasso
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.svm.classes import NuSVC
from sklearn.svm.classes import NuSVR
from sklearn.linear_model.omp import OrthogonalMatchingPursuit
from sklearn.cross_decomposition.pls_ import PLSCanonical
from sklearn.cross_decomposition.pls_ import PLSRegression
from sklearn.linear_model.passive_aggressive import PassiveAggressiveClassifier
from sklearn.linear_model.passive_aggressive import PassiveAggressiveRegressor
from sklearn.linear_model.perceptron import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model.ransac import RANSACRegressor
from sklearn.neighbors.classification import RadiusNeighborsClassifier
from sklearn.neighbors.regression import RadiusNeighborsRegressor
from sklearn.linear_model.ridge import Ridge
from sklearn.linear_model.ridge import RidgeClassifier
from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.svm.classes import SVC
from sklearn.svm.classes import SVR
from sklearn.linear_model.theil_sen import TheilSenRegressor

In [4]:
mlts = [MultiOutputRegressor, ClassifierChain]
bss = [Ridge, LinearSVR, GaussianProcessRegressor, PassiveAggressiveRegressor, KNeighborsRegressor, MLPRegressor, HuberRegressor, SGDRegressor, LassoLarsIC, ExtraTreeRegressor, ElasticNet, DecisionTreeRegressor, Perceptron, OrthogonalMatchingPursuit, BernoulliNB]

In [5]:
tfidf = pandas.read_csv('07 NM PT TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
tfidf[:3]

,____________________,_____________________,_____________________ follow,able,acceptable,accessory,accessory require,accidentally,accommodate,accompany,...,work,would,write,writer,wrong,wrong patient,wrong tattoo,xrt,xrt date,xrt pt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
ohe = pandas.read_csv('07 NM PT OHE.csv', delimiter=',', encoding='latin-1').fillna('')
ohe[:3]

,Wrong target or OAR contours,Treatment plan acceptable but not physically deliverable,Excess imaging dose,Failure to perform on-treatment imaging as per instructions,Wrong patient,Fall or other patient injury or medical condition,Wrong anatomical site (excluding laterality),Treatment plan (isodose distribution) unacceptable,Systematic hardware/software (including dose-volume) error,Interventional procedure error (Retired value),...,Inappropriate or poorly informed decision to treat or plan,Inadequate coordination of combined modality care,"Wrong, missing, mislabeled, or damaged treatment accessories",Wrong side (laterality),Wrong prescription dose-fractionation or calculation error,Radiation therapy scheduling error,Treatment not delivered - personnel/hardware/software failure,"Wrong patient position, setup point, or shift",Wrong planning margins,Allergic reaction
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
def lowest_correct(trues, preds):
    num_of_options = len(trues)
    drop_down_options = list(reversed(numpy.argsort(preds)))
    correct_options = [i for i in range(num_of_options) if trues[i]==1]
    return max([drop_down_options.index(correct_option) for correct_option in correct_options]) + 1
def average_lowest_correct(list_of_trues, list_of_preds):
    length = len(list_of_trues)
    return numpy.mean([lowest_correct(list(list_of_trues.iloc[i]), list(list_of_preds[i])) for i in range(length)])

## Default

In [8]:
def deadline(timeout, *args):
    def decorate(f):
        def handler(signum, frame):
            raise Exception

        def new_f(*args):
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(timeout)
            return f(*args)
            signal.alarm(0)

        new_f.__name__ = f.__name__
        return new_f
    return decorate

In [9]:
@deadline(500)
def cv_ensemble(model, xs, ys):
    temp = cross_validate(model, xs, ys, return_train_score=True, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2)
    return numpy.mean(temp['test_score']).round(2)

In [14]:
cv_ensemble(MultiOutputRegressor(PassiveAggressiveRegressor()), tfidf, ohe)

4.37

## Grid Search

In [10]:
def prn_rsl(gsc):
    prms = gsc.cv_results_['params']
    tsts = (gsc.cv_results_['split0_test_score']+gsc.cv_results_['split1_test_score'])/2
    for i in numpy.argsort(tsts):
        print(prms[i], tsts[i])

In [11]:
ppl = Pipeline([('mb', MultiOutputRegressor(PassiveAggressiveRegressor()))])

In [16]:
grd = {
    'mb__estimator__fit_intercept': [True, False],
    'mb__estimator__average': [True, False],
    'mb__estimator__loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2, verbose=0, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

{'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive'} 3.2326228499020435
{'mb__estimator__average': True, 'mb__estimator__fit_intercept': False, 'mb__estimator__loss': 'squared_epsilon_insensitive'} 3.371394598019304
{'mb__estimator__average': False, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive'} 3.4722043109801035
{'mb__estimator__average': True, 'mb__estimator__fit_intercept': False, 'mb__estimator__loss': 'epsilon_insensitive'} 3.564847808113306
{'mb__estimator__average': False, 'mb__estimator__fit_intercept': False, 'mb__estimator__loss': 'squared_epsilon_insensitive'} 3.6983959640167514
{'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'epsilon_insensitive'} 3.7080247856642163
{'mb__estimator__average': False, 'mb__estimator__fit_intercept': False, 'mb__estimator__loss': 'epsilon_insensitive'} 3.893456130812229
{'mb__estima

In [14]:
grd = {
    'mb__estimator__fit_intercept': [True],
    'mb__estimator__average': [True],
    'mb__estimator__loss': ['squared_epsilon_insensitive'],
    'mb__estimator__C': [0.1**i for i in range(-2, 3)],
    'mb__estimator__max_iter': [i for i in range(2, 10, 2)],
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2, verbose=0, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

{'mb__estimator__C': 0.1, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 4} 2.7477701439689417
{'mb__estimator__C': 0.1, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 6} 2.795990078544853
{'mb__estimator__C': 0.1, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 2} 2.805059470316516
{'mb__estimator__C': 0.1, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 8} 2.820890324783867
{'mb__estimator__C': 1.0, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 2} 2.9062191077879826
{'mb__es

In [15]:
grd = {
    'mb__estimator__fit_intercept': [True],
    'mb__estimator__average': [True],
    'mb__estimator__loss': ['squared_epsilon_insensitive'],
    'mb__estimator__C': [0.1**i for i in range(-2, 3)],
    'mb__estimator__max_iter': [i for i in range(10, 100, 10)],
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2, verbose=0, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

{'mb__estimator__C': 0.010000000000000002, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 30} 2.734547715549005
{'mb__estimator__C': 0.010000000000000002, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 20} 2.7414602692452865
{'mb__estimator__C': 0.010000000000000002, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 40} 2.7811651868360983
{'mb__estimator__C': 0.010000000000000002, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 50} 2.8065939707388967
{'mb__estimator__C': 0.010000000000000002, 'mb__estimator__average': True, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss

In [16]:
grd = {
    'mb__estimator__fit_intercept': [True],
    'mb__estimator__average': [True],
    'mb__estimator__loss': ['squared_epsilon_insensitive'],
    'mb__estimator__C': [0.1],
    'mb__estimator__max_iter': [4],
    'mb__estimator__tol': [0.1**i for i in range(1,6)],
    'mb__estimator__epsilon': [0.1**i for i in range(-2, 2)],
    
}
gsc = GridSearchCV(estimator=ppl, param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2, verbose=0, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

{'mb__estimator__C': 0.1, 'mb__estimator__average': True, 'mb__estimator__epsilon': 0.1, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 4, 'mb__estimator__tol': 1.0000000000000003e-05} 2.724474046048493
{'mb__estimator__C': 0.1, 'mb__estimator__average': True, 'mb__estimator__epsilon': 0.1, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 4, 'mb__estimator__tol': 0.00010000000000000002} 2.734013561119399
{'mb__estimator__C': 0.1, 'mb__estimator__average': True, 'mb__estimator__epsilon': 0.1, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 4, 'mb__estimator__tol': 0.1} 2.735103213688732
{'mb__estimator__C': 0.1, 'mb__estimator__average': True, 'mb__estimator__epsilon': 0.1, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimato

### Default
4.37
### Tuned
{'mb__estimator__C': 0.1, 'mb__estimator__average': True, 'mb__estimator__epsilon': 0.1, 'mb__estimator__fit_intercept': True, 'mb__estimator__loss': 'squared_epsilon_insensitive', 'mb__estimator__max_iter': 4, 'mb__estimator__tol': 1.0000000000000003e-05} 

2.72